# Funcation Call进阶技巧

## 1. Funcation Call调用流程回顾

### 1.1 Funcation call调用详细流程

In [201]:
import os
import openai
from openai import OpenAI
import shutil

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re
import random
import string

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")
#api_key="xxx"
## pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

- 定义了一个大模型里面肯定没有的算法

In [2]:
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())

In [3]:
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})
df

,x1,x2
0,1,3
1,2,4


- 与大模型进行配合调用的时候，尽可能的让调用的函数参数和返回值都用String表示。

In [4]:
df_str = df.to_string()
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [5]:
result_json=sunwukong_function(df_str)
result_json

'"   x1  x2\\n0  10  30\\n1  20  40"'

- 定义函数的说明，方便大模型识别函数自动完成调用，注意格式为：JSON Schema的格式

In [6]:
sunwukong={
        "type": "function",
        "function": {"name": "sunwukong_function",
                      "description": "用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行孙悟空算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }
    }

In [7]:
tools = [sunwukong]

- 在没有外部函数工具的情况下测试大模型的调用并执行孙悟空算法

In [8]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}
]

response =  client.chat.completions.create(
  model="glm-4",
  messages=messages
)
response.choices[0].message

CompletionMessage(content='很抱歉，但我需要更多的上下文信息来理解你所说的"孙悟空算法"是什么。在数据科学和机器学习的领域，我没有听说过这个名字的算法。\n\n然而，如果你提供的数据集是一个简单的二维数组，并以字符串形式表示，我可以帮你解释如何将其转换为Python中的Pandas DataFrame，然后你可以应用任何你想要的算法进行分析。\n\n以下是一个简单的例子，说明如何将你提供的数据集字符串转换为Pandas DataFrame：\n\n```python\nimport pandas as pd\n\n# 假设这是你提供的数据集字符串\ndata_str = """x1  x2\n0   1   3\n1   2   4"""\n\n# 使用Pandas的read_csv函数将其转换为DataFrame，需要指定分隔符（此处为空格）\ndf = pd.read_csv(pd.compat.StringIO(data_str), sep="\\s+")\n\nprint(df)\n```\n\n上面的代码会输出：\n\n```\n   x1  x2\n0   0   1\n1   1   2\n2   3   4\n```\n\n注意：这里有两个假设：\n1. 每个数据点之间由一个或多个空格分隔。\n2. 第一行包含列名，其余行包含数据。\n\n一旦你有了DataFrame，你可以应用任何数据分析或机器学习算法，比如统计计算、预测模型等。\n\n如果你能提供更多关于"孙悟空算法"的信息，我会尽力帮助你。', role='assistant', tool_calls=None)

- 测试Funcation call流程

第一次调用大模型

In [9]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}
]

In [10]:


response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
        tools=tools,
        tool_choice="auto",  
    )

response.choices[0].message

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8730210251242354639', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function', index=0)])

- 模型识别正确识别出来需要调用的函数，并且正确解析了参数

In [11]:
first_response = response.choices[0].message
first_response

CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8730210251242354639', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function', index=0)])

In [12]:
response.choices[0].message.tool_calls

[CompletionMessageToolCall(id='call_8730210251242354639', function=Function(arguments='{"data":"x1  x2\\n0   1   3\\n1   2   4"}', name='sunwukong_function'), type='function', index=0)]

- 构建字典形式的函数调用列别，key为函数名，value为函数体

In [13]:
available_tools =  {
    "sunwukong_function": sunwukong_function,
}

- 根据大模型的返回值中获取到函数调用需要到的：函数名，参数，函数体，并且执行函数。

In [14]:
tool_calls = response.choices[0].message.tool_calls
 
for tool_call in tool_calls:
    ## 函数名
    function_name = tool_call.function.name
    ## 根据函数名获取到函数体
    function_to_call = available_tools[function_name]
    function_args = json.loads(tool_call.function.arguments)
    ## 执行函数
    function_response = function_to_call(**function_args)
 

print(function_name)
print(function_args)
print(function_response)

sunwukong_function
{'data': 'x1  x2\n0   1   3\n1   2   4'}
"   x1  x2\n0  10  30\n1  20  40"


In [15]:
function_response = function_to_call(**function_args)
function_response

'"   x1  x2\\n0  10  30\\n1  20  40"'

第二次调用大模型

- 把结果给到大模型，大模型进行增强生成

In [16]:
messages.append(response.choices[0].message.model_dump())

In [19]:
for tool_call in tool_calls:
    
    function_name = tool_call.function.name
    function_to_call = available_tools[function_name]
    function_args = json.loads(tool_call.function.arguments)
    ## 真正执行外部函数的就是这儿的代码
    function_response = function_to_call(**function_args)
    messages.append(
        {
            "role": "tool",
            "content": function_response,
            "tool_call_id": tool_call.id,
        }
    ) 
## 第二次调用模型
second_response = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools
    ) 
# 获取最终结果
final_response = second_response.choices[0].message.content

In [20]:
final_response

'您好，根据您提供的数据集，经过孙悟空算法的处理，得到的结果如上所示。我们可以看到，x1和x2的值都乘以了10。这是孙悟空算法的一种特殊计算过程。如有需要，您可以根据这个结果进行后续的分析和处理。'

### 1.2 流程封装

In [32]:
from openai import OpenAI
import json

## 步骤1：构建函数
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())


available_tools =  {
    "sunwukong_function": sunwukong_function,
}


df_str=pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]}).to_string

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法"}  
        ]
    ## 步骤2：函数说明  JSON Schema格式
    tools = [
    {
        "type": "function",
        "function": {"name": "sunwukong_function",
                      "description": "用于执行孙悟空算法函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行孙悟空算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }
    }]
    ## 步骤3：第一次调大模型
    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        messages.append(response.choices[0].message.model_dump())
     
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append({
                "role": "tool",
                "content": f"{json.dumps(function_response)}",
                "tool_call_id":tool_call.id
            })
        ## 步骤四：第二次调用大模型
        second_response = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=messages,
            tools=tools,
        )
        return second_response
    
result=run_conversation()

In [34]:
result.choices[0].message.content

'根据您的需求，我已成功在数据集data上执行了孙悟空算法，并将结果以字符串形式返回。经过计算，数据集变为："   x1  x2\\n0  10  30\\n1  20  40"。'

## 2. 挑战1-意图识别

### 2.1 问题测试

In [ ]:
auto_functions是一个自动帮我们生成tools工具描述的方法

In [49]:
import inspect
import json
import pandas as pd
from typing import List, Callable

def auto_functions(func_list: List[Callable]) -> List[dict]:
    tools_description = []
    
    for func in func_list:
        # 获取函数的签名信息
        sig = inspect.signature(func)
        func_params = sig.parameters
        
        # 函数的参数描述
        parameters = {
            'type': 'object',
            'properties': {},
            'required': []
        }
        
        for param_name, param in func_params.items():
            # 添加参数描述和类型
            parameters['properties'][param_name] = {
                'description': param.annotation.__doc__ if param.annotation is not inspect._empty else "",
                'type': str(param.annotation) if param.annotation != param.empty else 'Any'
            }
            # 如果参数有默认值，那么它不是必须的
            if param.default != param.empty:
                parameters['required'].append(param_name)
        
        # 函数描述字典
        func_dict = {
            "type": "function",
            "function": {
                "name": func.__name__,
                "description": func.__doc__.strip(),
                "parameters": parameters
            }
        }
        
        tools_description.append(func_dict)
    
    return tools_description

In [37]:
## 打印大模型识别到的外部函数的名字
def function_call_test(prompt,tools):
    prompt = prompt
    message = [
        {"role": "user", "content": prompt}
    ]
    
    response1 = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = "auto"
        )

    print(response1.choices[0].message.tool_calls[0].function.name)

In [38]:
def machine_learning_1():
    """
    解释机器学习是什么
    """

    answer = """机器学习是人工智能的一个分支，研究计算机如何自动从数据中学习，提升性能并做出预测。\
    它通过算法让计算机提炼知识，优化任务执行，而无需明确编程。"""
    
    return answer

def machine_learning_2():
    
    """描述一下机器学习和深度学习的区别"""
    
    answer = """机器学习涉及算法从数据中学习模式以做出预测或决策，涵盖广泛的技术，包括监督、非监督和强化学习。\
    深度学习是机器学习的一个子集，专注于使用神经网络，尤其是深层神经网络，来处理复杂的数据结构，\
    实现高级功能如图像识别和自然语言理解，其特点是多层次的抽象和自动特征学习。"""
    
    return answer

In [50]:
# 函数列表
functions_list = [machine_learning_1, machine_learning_2]
# 获取函数描述的 JSON 字典
tools = auto_functions(functions_list)
# 将函数列表转换为字典
available_functions = {func.__name__: func for func in functions_list}

In [51]:
tools

[{'type': 'function',
  'function': {'name': 'machine_learning_1',
   'description': '解释机器学习是什么',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}},
 {'type': 'function',
  'function': {'name': 'machine_learning_2',
   'description': '描述一下机器学习和深度学习的区别',
   'parameters': {'type': 'object', 'properties': {}, 'required': []}}}]

In [54]:
available_functions

{'machine_learning_1': <function __main__.machine_learning_1()>,
 'machine_learning_2': <function __main__.machine_learning_2()>}

In [56]:
prompt = "请问机器学习是什么？"
function_call_test(prompt,tools)

machine_learning_1


In [57]:
prompt = "请问深度学习和机器学习的区别是什么？"
function_call_test(prompt,tools)

machine_learning_2


In [58]:
prompt = "请问深度学习是什么？"
function_call_test(prompt,tools)

machine_learning_1


这个时候我们发现大模型调用错了函数，他认为机器学习和深度学习很像，类似的生产业务中也有类似的情况，比如在银行业务中，用户开卡和开信用卡，应该对应的是两个不同的函数。
但是如果类似上面这种，很有可能就是会被当做是同一个业务进行处理了。显然不对。

### 2.2 解决方案

**尽最大能力提供高质量的Json schema**

> 这里的“高质量”是指清晰地描述了函数的功能和参数、易于模型进行辨别、使用简洁且一致的命名约定、保持函数名格式相同、但是尽量不要使用相似或容易混淆的函数名称（比如前面的machine_learning_1和Machine_learning_2就是非常糟糕的例子），函数的description尽量不要使用过于相似的描述等等；**这个过程免不了人工干预，Json schema的质量越高，模型辨别也就越容易**。<br><br>
> 更具体的手段包括：
> > **1. 使用关键词**：从实验结果来看，GLM-4可能很大程度上是基于embedding后的结果的相似程度来选择函数的，prompt中如果出现与函数名称高度一致的语句片段，那个函数就有很大的概率被选中（例如，prompt是“深度学习是什么”，函数名是“机器学习是什么”，即便什么是深度学习这个知识点早就在GLM的知识库内，它还是会被function吸引走），因此对特定函数、特定需求进行“**关键词**”的设置异常重要，这个关键词可以是一些很奇妙的语言，在正常的对话中绝对不会出现混淆的语言，例如编号，或者特定的专有名词。存在特定的业务是一定要执行某个函数或某个流程的，你可以纯依赖于编程、而不依赖于大模型的理解来走完这个流程。<br><br>
> > **2. 使用特定词语对函数的作用进行分组和分类**：在函数名称或描述中添加分类标记。例如，可以在名称中使用前缀如“user_”、“admin_”或“data_”等，帮助模型快速定位相关类别的函数。<br><br>
> > **3. 为函数增加权重**：为使用更频繁的函数增加primary、prior、core等有助于模型理解其重要性的名称，也可以在进行编号或者函数的描述时写上“非常重要，经常使用，核心业务”等等描述；<br><br>
> > **4. 使用否定句，帮助模型进一步辨析函数功能等**：比如在函数的描述中，明确说明“当出现xxxx语言或prompt时，这个函数不管用，请谨慎考虑选择该函数”等。

In [59]:
prompt = "我来到你的城市，走过你来时的路，\
轻轻地我走了，正如我轻轻的来。\
你是我的小苹果，小呀小苹果。\
送你一朵小红花"

def 送你一朵小红花():
    """如函数名"""
    a = 1314520
    return a

def 我只是在唱歌():
    """如函数名"""
    a = 1314520
    return a

# 函数列表
functions_list2 = [送你一朵小红花,我只是在唱歌]
# 获取函数描述的 JSON 字典
tools2 = auto_functions(functions_list2)
# 将函数列表转换为字典
available_functions2 = {func.__name__: func for func in functions_list2}

In [60]:
function_call_test(prompt,tools2)

送你一朵小红花


**干预模型选择**

> 通过改变function call交互的流程、改变模型的选择；包括但不限于——
> > **使用自查prompts进行干预**：例如，在用户进行具体的提问时，使用“xxx是否在你的资料库内”prompt进行优先筛查，人为为模型增加一个“你是否知道xxx流程”的过程，可以避免模型在判断就是否需要调用函数时混淆。但需要注意的是，因为大模型会有幻觉现象，因此你的prompt要相当考究。如果你问模型“你是否知道xxx”，模型很大程度上会说谎。<br><br>
> > **依赖匹配好于依赖理解**：例如，准备某个业务下的关键词库，只要用户在提到这些关键词时，就指定tool_choice参数让GLM-4进行特定的函数的调用，或者在用户提到某些关键词时，就推荐模型使用某个特定的函数进行调用；一个典型的场景可以是，在餐厅场景中，用户说“我要点餐”、“我要吃饭”、“我要下单”、“我要xxx”、甚至“服务员过来一下”的语音都可能都是指向“点餐”的需求，可以先经过编程流程匹配，再交由模型进行理解。这个过程会让模型识别的精准程度大幅上升。

In [61]:
#在选择进行功能选择前，让模型先判断被提到的问题是否在它的资料库中
def function_call_new(prompt,tools):
    prompt = prompt
    judge_words = "这个问题的答案是否在你的语料库里？\
        请回答“这个问题答案在我的语料库里”或者“这个问题的答案不在我的语料库里”\
        不要回答其他额外的文字"
    message = [
        {"role": "user", "content": prompt + judge_words}
    ]

    response1 = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message
        )

    if "这个问题答案在我的语料库里" in response1.choices[0].message.content:
        message = [
        {"role": "user", "content": prompt}
    ]
        response2 = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=message,
        )

        print(response2.choices[0].message.content)

    else:
        message = [
        {"role": "user", "content": prompt}
    ]
        response2 = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=message,
            tools = tools,
            tool_choice = "auto"
            )
        print(response2.choices[0].message.tool_calls[0].function.name)

In [ ]:
def machine_learning_1():
    """
    解释机器学习是什么
    """

    answer = """机器学习是人工智能的一个分支，研究计算机如何自动从数据中学习，提升性能并做出预测。\
    它通过算法让计算机提炼知识，优化任务执行，而无需明确编程。"""
    
    return answer

def machine_learning_2():
    
    """描述一下机器学习和深度学习的区别"""
    
    answer = """机器学习涉及算法从数据中学习模式以做出预测或决策，涵盖广泛的技术，包括监督、非监督和强化学习。\
    深度学习是机器学习的一个子集，专注于使用神经网络，尤其是深层神经网络，来处理复杂的数据结构，\
    实现高级功能如图像识别和自然语言理解，其特点是多层次的抽象和自动特征学习。"""
    
    return answer

In [62]:
# 函数列表
functions_list = [machine_learning_1, machine_learning_2]
# 获取函数描述的 JSON 字典
tools = auto_functions(functions_list)
# 将函数列表转换为字典
available_functions = {func.__name__: func for func in functions_list}

In [63]:
prompt = "请问机器学习是什么？"
function_call_new(prompt,tools)

机器学习是一门人工智能（AI）的分支，它让计算机系统能从数据中学习并做出决策或预测，而无需进行显式编程。机器学习算法利用样本数据（称为训练数据）来识别数据中的模式，并基于这些模式做出推断或决策。简单地说，机器学习使得计算机能够基于经验改进其性能或行为。

机器学习可以分为几个主要类别：

1. 监督学习（Supervised Learning）：在这种模式下，算法从标记过的训练数据中学习，即每个样本都有一个对应的输出标签。目标是训练出一个模型，它能对新的、未见过的数据进行准确预测。

2. 无监督学习（Unsupervised Learning）：在此类别中，算法使用没有标记的数据进行学习，即数据没有附加的输出标签。无监督学习的目标是发现数据中的结构或模式，如通过聚类相似的数据点。

3. 强化学习（Reinforcement Learning）：这是一种学习策略，系统（称为“智能体”）在某个环境中采取行动，并根据行动的结果学习最佳行为。强化学习的目标是最大化累积奖励。

机器学习在各个领域都有广泛应用，包括但不限于金融、医疗保健、自动驾驶、推荐系统、语音和图像识别等。随着技术的发展，机器学习已经成为解决复杂问题的重要工具，并在我们的日常生活中发挥着越来越重要的作用。


In [ ]:
prompt = "请问深度学习和机器学习的区别是什么？"
function_call_new(prompt,tools)

In [64]:
prompt = "请问深度学习是什么？"
function_call_new(prompt,tools)

深度学习是机器学习的一个子领域，它模仿了人类大脑的神经网络结构和功能，以通过算法模型对数据进行特征学习和模式识别。在深度学习中，所谓的“深度”指的是神经网络的结构，它包含多个处理层，每一层都由许多神经元组成，这些层之间的关系可以是线性的也可以是非线性的。

在深度学习的模型中，每一层都会对输入数据进行处理，并将其结果传递到下一层。通过这种方式，模型能够从原始数据中自动提取更高层次的特征表示。这一过程不需要人工干预，区别于传统的机器学习方法，后者通常需要人工设计特征。

深度学习在多个领域展现了强大的能力，特别是在图像和语音识别、自然语言处理、医疗诊断以及自动驾驶技术等领域。其流行的主要原因是：

1. 大数据时代的到来提供了海量数据，这些数据为深度学习模型的训练提供了基础，使其能够学习到更加复杂和精细的模式。
2. 算法上的创新，如新的激活函数（如ReLU）、优化方法（如Adamax优化器）和损失函数（如交叉熵损失）等，都极大提高了神经网络的训练效果和效率。
3. 硬件技术的进步，例如GPU和TPU的广泛应用，为深度学习模型提供了强大的计算支持。

深度学习框架，如TensorFlow和PyTorch，提供了构建和训练复杂神经网络所需的工具和库，使得研究人员和开发者可以更加方便地实现深度学习模型。

综上所述，深度学习是一种强大的机器学习技术，它通过模仿人脑神经网络的工作机制，自动学习数据中的高级特征，并在各种复杂任务中展现出优异的性能。


### 2.3 问题测试

In [66]:
#以下是基于不同数学运算类型的七个函数示例：

def add(a: float, b: float) -> float:
    """
    计算两个浮点数的和。

    参数:
    a (float): 第一个加数。
    b (float): 第二个加数。

    返回:
    float: 两个参数的和。
    """
    return a + b

def subtract(a: float, b: float) -> float:
    """
    计算两个浮点数的差。

    参数:
    a (float): 被减数。
    b (float): 减数。

    返回:
    float: 两个参数的差。
    """
    return a - b

def multiply(a: float, b: float) -> float:
    """
    计算两个浮点数的乘积。

    参数:
    a (float): 第一个乘数。
    b (float): 第二个乘数。

    返回:
    float: 两个参数的乘积。
    """
    return a * b

def divide(a: float, b: float) -> float:
    """
    计算两个浮点数的商。

    参数:
    a (float): 被除数。
    b (float): 除数。注意：除数不能为0。

    返回:
    float: 两个参数的商。
    """
    if b == 0:
        raise ValueError("除数不能为0")
    return a / b

def power(a: float, b: float) -> float:
    """
    计算第一个浮点数的幂运算结果。

    参数:
    a (float): 底数。
    b (float): 指数。

    返回:
    float: 底数的指数幂。
    """
    return a ** b

In [67]:
# 你的函数列表
functions_list = [add,multiply,subtract,divide,power]

# 获取函数描述的 JSON 字典
tools3 = auto_functions(functions_list)

# 将函数列表转换为字典
available_functions = {func.__name__: func for func in functions_list}

In [70]:
def function_call(prompt,tools):
    prompt = prompt
    message = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = "auto"
        )
    print(response)
    
    #找出相应的函数，获取函数结果
    function_to_call = available_functions[response.choices[0].message.tool_calls[0].function.name]
    function_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    function_response = function_to_call(**function_args)

    message.append({
                "role": "tool",
                "name": response.choices[0].message.tool_calls[0].function.name,
                "content": prompt + str(function_response),
            })
    
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = response.choices[0].message.tool_calls[0].function.name,
        )
    return print(response.choices[0].message.content)

In [68]:
56565656 * 34343

1942634324008

In [75]:
for i in range(5):
    prompt = "请帮我计算一下 56565656 乘以 34343 等于几"
    function_call(prompt,tools3)
    print("\n===============================")

不对，这个结果太大了。让我重新计算一下。

56565656 乘以 34343 等于 1942634324。

抱歉，我之前的回答是错误的。让我重新计算一下：

56565656 乘以 34343 等于：

56565656 * 34343 = 1,942,634,332,848

所以，56565656 乘以 34343 的结果是 1,942,634,332,848。

抱歉，我之前的回答是错误的。让我重新计算一下：

56565656 乘以 34343 等于：

56565656 * 34343 = 1,942,634,332,848

所以，56565656 乘以 34343 的结果是 1,942,634,332,848。

The product of 56565656 and 34343 is 1942634324008.

这个问题可以通过使用编程语言中的整数乘法来解决。下面是使用Python语言的代码示例:

```
result = 56565656 * 34343
print(result)
```

执行这段代码,输出结果为:

```
1942634324008
```

因此,56565656乘以34343等于1942634324008。



In [78]:
def function_call_for_test(prompt,tools):
    """
    相比于之前的版本，增加了“究竟是否调用了函数的判断”，可以帮助我们打印是否调用了模型
    """
    prompt = prompt
    message = [
        {"role": "user", "content": prompt}
    ]
    
    response_f = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = "auto"
        )
    
    function_calls = response_f.choices[0].message.tool_calls[0]
    function_to_call = available_functions[function_calls.function.name]
    function_args = json.loads(function_calls.function.arguments)
    function_response = function_to_call(**function_args)

    message.append({
                "role": "tool",
                "name": response_f.choices[0].message.tool_calls[0].function.name,
                "content": prompt + str(function_response),
            })
    
    response_a = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = response_f.choices[0].message.tool_calls[0].function.name,
        )

    print(response_a.choices[0].message.content)
    
    #是否成功调用了函数呢？
    try:
        response_f.choices[0].message.tool_calls[0].function.name
        print(" ")
        print("\n被调用的函数是{}".format(response_f.choices[0].message.tool_calls[0].function.name))
    except:
        print(" ")
        print("\n没有函数被成功调用！")

In [77]:
56565656 * 34343

1942634324008

In [79]:
for i in range(5):
    prompt = "请帮我计算一下 56565656 乘以 34343 等于几"
    function_call_for_test(prompt,tools3)
    print("\n===============================")

抱歉，我之前的回答有误。让我重新计算一下：

56565656 乘以 34343 等于：

56565656 * 34343 = 1,942,634,332,848

所以，56565656 乘以 34343 的结果是 1,942,634,332,848。
 

被调用的函数是multiply

To calculate 56565656 multiplied by 34343, you can use a calculator or a programming function that handles large numbers. The result is:

56565656 * 34343 = 1,942,634,324,008

So, 56565656 multiplied by 34343 equals 1,942,634,324,008.
 

被调用的函数是multiply

The product of 56565656 and 34343 is 1942634324008.
 

被调用的函数是multiply

实际上，56565656 乘以 34343 的结果并不是 1942634324008。我们可以手动计算这个乘法来验证正确的答案。

56565656
x       34343
-----------
28282828
+ 1699328
+   565656
+     34343
-----------
190856630488

所以，56565656 乘以 34343 的正确结果是 190856630488。
 

被调用的函数是multiply

这个结果是错误的。让我们重新计算一次：

56565656 乘以 34343 等于 1,942,634,332,368。

这是正确的结果。
 

被调用的函数是multiply



我们发现都是成功的调用到了外部的函数，外部函数执行的结果肯定是没问题的。但是为什么还是结果有错呢？

In [ ]:
message.append({
                "role": "tool",
                "name": response_f.choices[0].message.tool_calls[0].function.name,
                "content": prompt + str(function_response),
            })
    
response_a = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = response_f.choices[0].message.tool_calls[0].function.name,
        )

print(response_a.choices[0].message.content)

这段代码遵循的是OpenAI早期的规则，即定义好角色tool之后，要分别定义好使用的函数的名字“name”和角色tool所携带的信息content，content中是原始的prompt（问题）+ 函数返回的答案function_response。在将这一部分信息放入模型的时候，tools工具必须保持打开状态，并且tool_choice工具需要给到具体的函数名称name。很显然，这套代码本身的作用是帮助大模型认识到原本的上下文信息、并且意识到现在的正确答案是通过function call功能获得的，然而从实际实验结果来看，这个将信息传回的过程却不是很有效。

如下是清华智谱官网，给的代码演示

In [ ]:

    tool_call = model_response.choices[0].message.tool_calls[0]
    args = tool_call.function.arguments
    function_result = get_flight_number(**json.loads(args))

    messages.append({
        "role": "tool",
        "content": f"{json.dumps(function_result)}",
        "tool_call_id":tool_call.id
    })

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
    )
    print(response.choices[0].message)
    messages.append(response.choices[0].message.model_dump())

和OpenAI的代码不同，这段代码中不是使用函数的名字，而是在messange中加入了函数被抽取出来的id，帮助GLM更好地定义到相应的函数功能上。然而，经过实验的结果，上述代码的结果比OpenAI官方给的代码返回结果还糟糕。由于tool_call.id中往往包含了大量随机字符（毕竟是标记这个函数的唯一ID），因此会出现模型的输出变成了英文、变成了代码、变成了乱码等等情况；同时，模型不仅不能很好地认知到函数功能返回了正确答案，还会返回与正确答案毫无关联的代码等等信息。因此，智谱AI官方提供在官网上的这段代码更不可用。

### 2.4 优化调整

In [81]:
def function_call_new2(prompt,tools):
    """
    相比于之前的版本，进行了两个调整——
    首先，增加了“究竟是否调用了函数的判断”，可以帮助我们打印是否调用了模型
    其次，放弃了tool角色，改用assistant角色传入结果，并增加prompts
    """
    prompt = prompt
    message = [
        {"role": "user", "content": prompt}
    ]
    
    response_f = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message,
        tools = tools,
        tool_choice = "auto"
        )
    
    function_calls = response_f.choices[0].message.tool_calls[0]
    function_to_call = available_functions[function_calls.function.name]
    function_args = json.loads(function_calls.function.arguments)
    function_response = function_to_call(**function_args)

    message = []
    message.append({
            "role": "assistant",
            "content": "你使用了tools工具，最终获得的答案是" + str(function_response),
        })
    message.append({"role": "user", "content": prompt + "请仅仅回答答案的数字，不要包括其他描述"})
    
    response_a = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=message
        )

    print(response_a.choices[0].message.content)
    
    #是否成功调用了函数呢？
    try:
        response_f.choices[0].message.tool_calls[0].function.name
        print(" ")
        print("\n被调用的函数是{}".format(response_f.choices[0].message.tool_calls[0].function.name))
    except:
        print(" ")
        print("\n没有函数被成功调用！")

In [82]:
56565656 * 34343

1942634324008

In [83]:
for i in range(5):
    prompt = "请帮我计算一下 56565656 乘以 34343 等于几"
    function_call_new2(prompt,tools3)
    print("\n===============================")

1942634324008
 

被调用的函数是multiply

1942634324008
 

被调用的函数是multiply

1942634324008
 

被调用的函数是multiply

1942634324008
 

被调用的函数是multiply

1942634324008
 

被调用的函数是multiply



我们发现这回5次全对！！

## 3 挑战2-海量函数调用

### 3.1 海量函数测试准备 

In [202]:
import math

def Basic_add(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算两个浮点数的和。

    参数:
    a (float): 第一个加数。
    b (float): 第二个加数。

    返回:
    float: 两个参数的和。
    """
    return a + b

def Basic_subtract(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算两个浮点数的差。

    参数:
    a (float): 被减数。
    b (float): 减数。

    返回:
    float: 两个参数的差。
    """
    return a - b

def Basic_divide(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算两个浮点数的商。

    参数:
    a (float): 被除数。
    b (float): 除数。

    返回:
    float: 两个参数的商。
    """
    if b == 0:
        raise ValueError("除数不能为零")
    return a / b

def Basic_mutiply(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算两个浮点数的乘积。

    参数:
    a (float): 第一个乘数。
    b (float): 第二个乘数。

    返回:
    float: 两个参数的乘积。
    """
    return a * b

def Basic_power(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算一个浮点数的幂。

    参数:
    a (float): 底数。
    b (float): 指数。

    返回:
    float: 底数的指数次幂。
    """
    return a ** b

def Basic_sqrt(a: float) -> float:
    """
    这是入门级别的数学计算。本函数计算一个浮点数的平方根。

    参数:
    a (float): 被开方数。

    返回:
    float: 被开方数的平方根。
    """
    if a < 0:
        raise ValueError("被开方数不能为负数")
    return a ** 0.5

def Basic_mod(a: float, b: float) -> float:
    """
    这是入门级别的数学计算。本函数计算两个浮点数的模。

    参数:
    a (float): 被除数。
    b (float): 除数。

    返回:
    float: 两个参数的模。
    """
    if b == 0:
        raise ValueError("除数不能为零")
    return a % b

def Basic_abs_val(a: float) -> float:
    """
    这是入门级别的数学计算。本函数计算一个浮点数的绝对值。

    参数:
    a (float): 输入浮点数。

    返回:
    float: 输入浮点数的绝对值。
    """
    return abs(a)

def Basic_factorial(a: int) -> int:
    """
    这是入门级别的数学计算。本函数计算一个整数的阶乘。

    参数:
    a (int): 输入整数。

    返回:
    int: 输入整数的阶乘。
    """
    if a < 0:
        raise ValueError("输入值不能为负数")
    result = 1
    for i in range(1, a + 1):
        result *= i
    return result

def Basic_gcd(a: int, b: int) -> int:
    """
    这是入门级别的数学计算。本函数计算两个整数的最大公约数。

    参数:
    a (int): 第一个整数。
    b (int): 第二个整数。

    返回:
    int: 两个整数的最大公约数。
    """
    while b:
        a, b = b, a % b
    return a

def Basic_lcm(a: int, b: int) -> int:
    """
    这是入门级别的数学计算。本函数计算两个整数的最小公倍数。

    参数:
    a (int): 第一个整数。
    b (int): 第二个整数。

    返回:
    int: 两个整数的最小公倍数。
    """
    if a == 0 or b == 0:
        return 0
    return abs(a * b) // gcd(a, b)

def Basic_exp(a: float) -> float:
    """
    这是入门级别的数学计算。本函数计算e的幂。

    参数:
    a (float): 指数。

    返回:
    float: e的a次幂。
    """
    import math
    return math.exp(a)

def Basic_log(a: float, base: float = 10) -> float:
    """
    这是入门级别的数学计算。本函数计算给定基数的对数。

    参数:
    a (float): 输入值。
    base (float): 对数的基数。

    返回:
    float: 输入值的对数。
    """
    import math
    if a <= 0:
        raise ValueError("输入值必须大于零")
    if base <= 0:
        raise ValueError("基数必须大于零")
    return math.log(a, base)

def Basic_log10(a: float) -> float:
    """
    这是入门级别的数学计算。本函数计算以10为底的对数。

    参数:
    a (float): 输入值。

    返回:
    float: 输入值的以10为底的对数。
    """
    import math
    if a <= 0:
        raise ValueError("输入值必须大于零")
    return math.log10(a)

def Basic_round_val(a: float, digits: int = 0) -> float:
    """
    这是入门级别的数学计算。本函数对浮点数进行四舍五入。

    参数:
    a (float): 输入浮点数。
    digits (int): 保留的小数位数。

    返回:
    float: 四舍五入后的浮点数。
    """
    return round(a, digits)

def Basic_level_ceil(a: float) -> int:
    """
    这是入门级别的数学计算。本函数向上取整一个浮点数。

    参数:
    a (float): 输入浮点数。

    返回:
    int: 向上取整后的整数。
    """
    import math
    return math.ceil(a)

def Basic_level_floor(a: float) -> int:
    """
    这是入门级别的数学计算。本函数向下取整一个浮点数。

    参数:
    a (float): 输入浮点数。

    返回:
    int: 向下取整后的整数。
    """
    import math
    return math.floor(a)

def Tri_func_sin_val(a: float) -> float:
    
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的正弦值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的正弦值。
    """
    import math
    return math.sin(a)

def Tri_func_cos_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的余弦值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的余弦值。
    """
    import math
    return math.cos(a)

def Tri_func_tan_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的正切值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的正切值。
    """
    import math
    return math.tan(a)

def Tri_func_asin_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个值的反正弦值。

    参数:
    a (float): 输入值。

    返回:
    float: 输入值的反正弦值，以弧度为单位。
    """
    import math
    return math.asin(a)

def Tri_func_acos_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个值的反余弦值。

    参数:
    a (float): 输入值。

    返回:
    float: 输入值的反余弦值，以弧度为单位。
    """
    import math
    return math.acos(a)

def Tri_func_atan_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个值的反正切值。

    参数:
    a (float): 输入值。

    返回:
    float: 输入值的反正切值，以弧度为单位。
    """
    import math
    return math.atan(a)

def Tri_func_sinh_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的双曲正弦值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的双曲正弦值。
    """
    import math
    return math.sinh(a)

def Tri_func_cosh_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的双曲余弦值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的双曲余弦值。
    """
    import math
    return math.cosh(a)

def Tri_func_tanh_val(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数计算一个角度的双曲正切值。

    参数:
    a (float): 输入角度，以弧度为单位。

    返回:
    float: 角度的双曲正切值。
    """
    import math
    return math.tanh(a)

def Tri_func_degrees(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数将弧度转换为角度。

    参数:
    a (float): 输入弧度。

    返回:
    float: 转换后的角度。
    """
    import math
    return math.degrees(a)

def Tri_func_radians(a: float) -> float:
    """
    这是涉及三角函数的数学计算，本函数将角度转换为弧度。

    参数:
    a (float): 输入角度。

    返回:
    float: 转换后的弧度。
    """
    import math
    return math.radians(a)

def Prob_comb(n: int, k: int) -> int:
    """
    概率论相关计算，计算组合数C(n, k)。

    参数:
    n (int): 总数。
    k (int): 选择数。

    返回:
    int: 组合数C(n, k)。
    """
    import math
    return math.comb(n, k)

def Prob_perm(n: int, k: int) -> int:
    """
    概率论相关计算，计算排列数P(n, k)。

    参数:
    n (int): 总数。
    k (int): 选择数。

    返回:
    int: 排列数P(n, k)。
    """
    import math
    return math.perm(n, k)

def Prob_factorial_prob(n: int) -> float:
    """
    概率论相关计算，计算n的阶乘概率。

    参数:
    n (int): 输入整数。

    返回:
    float: n的阶乘概率。
    """
    import math
    return math.factorial(n)

def Prob_binom_pmf(n: int, k: int, p: float) -> float:
    """
    概率论相关计算，计算二项分布的概率质量函数。

    参数:
    n (int): 试验次数。
    k (int): 成功次数。
    p (float): 单次试验成功的概率。

    返回:
    float: 二项分布的概率质量函数值。
    """
    import math
    comb = math.comb(n, k)
    return comb * (p ** k) * ((1 - p) ** (n - k))

def Prob_binom_cdf(n: int, k: int, p: float) -> float:
    """
    概率论相关计算，计算二项分布的累积分布函数。

    参数:
    n (int): 试验次数。
    k (int): 成功次数。
    p (float): 单次试验成功的概率。

    返回:
    float: 二项分布的累积分布函数值。
    """
    import math
    cdf = 0
    for i in range(k + 1):
        cdf += binom_pmf(n, i, p)
    return cdf

def Prob_poisson_pmf(k: int, lam: float) -> float:
    """
    概率论相关计算，计算泊松分布的概率质量函数。

    参数:
    k (int): 事件数。
    lam (float): 平均事件发生率。

    返回:
    float: 泊松分布的概率质量函数值。
    """
    import math
    return (lam ** k) * math.exp(-lam) / math.factorial(k)

def Prob_poisson_cdf(k: int, lam: float) -> float:
    """
    概率论相关计算，计算泊松分布的累积分布函数。

    参数:
    k (int): 事件数。
    lam (float): 平均事件发生率。

    返回:
    float: 泊松分布的累积分布函数值。
    """
    import math
    cdf = 0
    for i in range(k + 1):
        cdf += poisson_pmf(i, lam)
    return cdf

def Prob_uniform_pdf(a: float, b: float, x: float) -> float:
    """
    概率论相关计算，计算均匀分布的概率密度函数。

    参数:
    a (float): 均匀分布的下限。
    b (float): 均匀分布的上限。
    x (float): 自变量。

    返回:
    float: 均匀分布的概率密度函数值。
    """
    import math
    if a <= x <= b:
        return 1 / (b - a)
    return 0

def Prob_uniform_cdf(a: float, b: float, x: float) -> float:
    """
    概率论相关计算，计算均匀分布的累积分布函数。

    参数:
    a (float): 均匀分布的下限。
    b (float): 均匀分布的上限。
    x (float): 自变量。

    返回:
    float: 均匀分布的累积分布函数值。
    """
    import math
    if x < a:
        return 0
    elif x > b:
        return 1
    else:
        return (x - a) / (b - a)

def Prob_normal_pdf(mu: float, sigma: float, x: float) -> float:
    """
    概率论相关计算，计算正态分布的概率密度函数。

    参数:
    mu (float): 均值。
    sigma (float): 标准差。
    x (float): 自变量。

    返回:
    float: 正态分布的概率密度函数值。
    """
    import math
    return (1 / (sigma * math.sqrt(2 * math.pi))) * math.exp(-0.5 * ((x - mu) / sigma) ** 2)

def Prob_normal_cdf(mu: float, sigma: float, x: float) -> float:
    """
    概率论相关计算，计算正态分布的累积分布函数。

    参数:
    mu (float): 均值。
    sigma (float): 标准差。
    x (float): 自变量。

    返回:
    float: 正态分布的累积分布函数值。
    """
    import math
    import scipy.stats as stats
    return stats.norm.cdf(x, loc=mu, scale=sigma)

def Prob_expon_pdf(lam: float, x: float) -> float:
    """
    概率论相关计算，计算指数分布的概率密度函数。

    参数:
    lam (float): 参数。
    x (float): 自变量。

    返回:
    float: 指数分布的概率密度函数值。
    """
    import math
    if x < 0:
        return 0
    return lam * math.exp(-lam * x)

def Prob_expon_cdf(lam: float, x: float) -> float:
    """
    概率论相关计算，计算指数分布的累积分布函数。

    参数:
    lam (float): 参数。
    x (float): 自变量。

    返回:
    float: 指数分布的累积分布函数值。
    """
    import math
    if x < 0:
        return 0
    return 1 - math.exp(-lam * x)

def Prob_geom_pmf(p: float, k: int) -> float:
    """
    概率论相关计算，计算几何分布的概率质量函数。

    参数:
    p (float): 单次试验成功的概率。
    k (int): 试验次数。

    返回:
    float: 几何分布的概率质量函数值。
    """
    import math
    if k < 1:
        return 0
    return (1 - p) ** (k - 1) * p

def Prob_geom_cdf(p: float, k: int) -> float:
    """
    概率论相关计算，计算几何分布的累积分布函数。

    参数:
    p (float): 单次试验成功的概率。
    k (int): 试验次数。

    返回:
    float: 几何分布的累积分布函数值。
    """
    import math
    if k < 1:
        return 0
    return 1 - (1 - p) ** k

def Prob_beta_pdf(alpha: float, beta: float, x: float) -> float:
    """
    概率论相关计算，计算Beta分布的概率密度函数。

    参数:
    alpha (float): 形状参数α。
    beta (float): 形状参数β。
    x (float): 自变量。

    返回:
    float: Beta分布的概率密度函数值。
    """
    import math
    import scipy.special as sp
    if x < 0 or x > 1:
        return 0
    return (x ** (alpha - 1)) * ((1 - x) ** (beta - 1)) / sp.beta(alpha, beta)

def Prob_beta_cdf(alpha: float, beta: float, x: float) -> float:
    """
    概率论相关计算，计算Beta分布的累积分布函数。

    参数:
    alpha (float): 形状参数α。
    beta (float): 形状参数β。
    x (float): 自变量。

    返回:
    float: Beta分布的累积分布函数值。
    """
    import math
    import scipy.stats as stats
    return stats.beta.cdf(x, alpha, beta)

def Advanced_Math_Derivative(func, a: float, h: float = 1e-5) -> float:
    """
    高等数学/微积分相关计算，计算函数在某点的导数。

    参数:
    func (function): 函数。
    a (float): 点。
    h (float): 微小变化量。

    返回:
    float: 函数在该点的导数。
    """
    import math
    return (func(a + h) - func(a - h)) / (2 * h)

def Advanced_Math_integral(func, a: float, b: float, n: int = 1000) -> float:
    """
    高等数学/微积分相关计算，计算函数在给定区间的定积分。

    参数:
    func (function): 函数。
    a (float): 积分下限。
    b (float): 积分上限。
    n (int): 分割区间数量。

    返回:
    float: 函数在给定区间的定积分。
    """
    import math
    width = (b - a) / n
    total = 0.5 * (func(a) + func(b))
    for i in range(1, n):
        total += func(a + i * width)
    return total * width

def Advanced_Math_double_integral(func, ax: float, bx: float, ay: float, by: float, nx: int = 1000, ny: int = 1000) -> float:
    """
    高等数学/微积分相关计算，计算二重积分。

    参数:
    func (function): 函数。
    ax (float): x方向积分下限。
    bx (float): x方向积分上限。
    ay (float): y方向积分下限。
    by (float): y方向积分上限。
    nx (int): x方向分割区间数量。
    ny (int): y方向分割区间数量。

    返回:
    float: 二重积分值。
    """
    import math
    width_x = (bx - ax) / nx
    width_y = (by - ay) / ny
    total = 0
    for i in range(nx):
        for j in range(ny):
            total += func(ax + i * width_x, ay + j * width_y)
    return total * width_x * width_y

def Advanced_Math_partial_derivative(func, var: int, point: list, h: float = 1e-5) -> float:
    """
    高等数学/微积分相关计算，计算函数在某点的偏导数。

    参数:
    func (function): 函数。
    var (int): 自变量的索引。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    float: 函数在该点的偏导数。
    """
    import math
    point_forward = point.copy()
    point_backward = point.copy()
    point_forward[var] += h
    point_backward[var] -= h
    return (func(*point_forward) - func(*point_backward)) / (2 * h)

def Advanced_Math_gradient(func, point: list, h: float = 1e-5) -> list:
    """
    高等数学/微积分相关计算，计算函数在某点的梯度。

    参数:
    func (function): 函数。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    list: 函数在该点的梯度。
    """
    import math
    return [partial_derivative(func, i, point, h) for i in range(len(point))]

def Advanced_Math_laplacian(func, point: list, h: float = 1e-5) -> float:
    """
    高等数学/微积分相关计算，计算函数在某点的拉普拉斯算子。

    参数:
    func (function): 函数。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    float: 函数在该点的拉普拉斯算子。
    """
    import math
    return sum(partial_derivative(func, i, point, h) for i in range(len(point)))

def Advanced_Math_divergence(vector_func, point: list, h: float = 1e-5) -> float:
    """
    高等数学/微积分相关计算，计算向量场在某点的散度。

    参数:
    vector_func (function): 向量场函数。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    float: 向量场在该点的散度。
    """
    import math
    return sum(partial_derivative(lambda *args: vector_func(*args)[i], i, point, h) for i in range(len(point)))

def Advanced_Math_curl(vector_func, point: list, h: float = 1e-5) -> list:
    """
    高等数学/微积分相关计算，计算向量场在某点的旋度。

    参数:
    vector_func (function): 向量场函数。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    list: 向量场在该点的旋度。
    """
    import math
    dF_dy = partial_derivative(lambda *args: vector_func(*args)[2], 1, point, h) - partial_derivative(lambda *args: vector_func(*args)[1], 2, point, h)
    dF_dz = partial_derivative(lambda *args: vector_func(*args)[0], 2, point, h) - partial_derivative(lambda *args: vector_func(*args)[2], 0, point, h)
    dF_dx = partial_derivative(lambda *args: vector_func(*args)[1], 0, point, h) - partial_derivative(lambda *args: vector_func(*args)[0], 1, point, h)
    return [dF_dy, dF_dz, dF_dx]

def Advanced_Math_jacobian(funcs, point: list, h: float = 1e-5) -> list:
    """
    高等数学/微积分相关计算，计算函数在某点的雅可比矩阵。

    参数:
    funcs (list): 函数列表。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    list: 雅可比矩阵。
    """
    import math
    return [[partial_derivative(funcs[i], j, point, h) for j in range(len(point))] for i in range(len(funcs))]

def Advanced_Math_hessian(func, point: list, h: float = 1e-5) -> list:
    """
    高等数学/微积分相关计算，计算函数在某点的黑塞矩阵。

    参数:
    func (function): 函数。
    point (list): 点。
    h (float): 微小变化量。

    返回:
    list: 黑塞矩阵。
    """
    import math
    n = len(point)
    return [[partial_derivative(lambda *args: partial_derivative(func, i, args, h), j, point, h) for j in range(n)] for i in range(n)]

def Advanced_Math_taylor_series(func, a: float, n: int) -> float:
    """
    高等数学/微积分相关计算，计算泰勒级数的前n项。
    注意与泰勒展开进行区别！
    泰勒级数是无穷级数、当题目中涉及到无穷的估计时，我们应该使用泰勒级数。当题目中涉及到有限项的展开时，我们应该使用n阶泰勒展开函数。

    参数:
    func (function): 函数。
    a (float): 展开点。
    n (int): 项数。

    返回:
    float: 泰勒级数的前n项。
    """
    import math
    series = 0
    for i in range(n):
        series += (derivative(func, a, h=1e-5) ** i) / math.factorial(i)
    return series

def Advanced_Math_taylor_expansion(func, a: float, n: int, x_val: float = None) -> str:
    """
    高等数学/微积分相关计算，计算函数在某点的泰勒展开（n阶展开）。

    注意与泰勒级数进行区别！
    泰勒级数是无穷级数、当题目中涉及到无穷的估计时，我们应该使用泰勒级数。当题目中涉及到有限项的展开时，我们应该使用n阶泰勒展开函数。

    参数:
    func (function): 要展开的函数。
    a (float): 展开点。
    n (int): 展开的阶数。
    x_val (float, optional): 如果提供，则计算泰勒展开在该点的具体值。

    返回:
    str: 泰勒展开的多项式表达式（字符串形式），如果提供了x_val，则返回泰勒展开在该点的具体值。
    """
    
    import math
    from sympy import Symbol, diff, exp
    
    # 定义符号
    x = Symbol('x')
    
    # 计算各阶导数
    terms = []
    for i in range(n + 1):
        term = diff(func(x), x, i).subs(x, a) / math.factorial(i) * (x - a)**i
        terms.append(term)
    
    # 合并多项式
    taylor_expansion = sum(terms)
    
    if x_val is not None:
        # 计算泰勒展开在x_val点的具体值
        taylor_value = taylor_expansion.subs(x, x_val)
        return str(taylor_value)
    
    return str(taylor_expansion)

def Advanced_Math_fourier_series(func, n: int, L: float = math.pi) -> list:
    """
    高等数学/微积分相关计算，计算函数的傅里叶级数。

    参数:
    func (function): 函数。
    n (int): 项数。
    L (float): 周期的一半。

    返回:
    list: 傅里叶级数系数。
    """
    import math
    a0 = (1 / L) * integral(func, -L, L, 1000)
    a = [(1 / L) * integral(lambda x: func(x) * math.cos((i * math.pi * x) / L), -L, L, 1000) for i in range(1, n + 1)]
    b = [(1 / L) * integral(lambda x: func(x) * math.sin((i * math.pi * x) / L), -L, L, 1000) for i in range(1, n + 1)]
    return [a0, a, b]

def Advanced_Math_laplace_transform(func, s: float) -> float:
    """
    高等数学/微积分相关计算，计算拉普拉斯变换。

    参数:
    func (function): 函数。
    s (float): 自变量。

    返回:
    float: 拉普拉斯变换值。
    """
    import math
    return integral(lambda t: func(t) * math.exp(-s * t), 0, math.inf, 1000)

def Advanced_Math_fourier_transform(func, w: float) -> float:
    """
    高等数学/微积分相关计算，计算傅里叶变换。

    参数:
    func (function): 函数。
    w (float): 自变量。

    返回:
    float: 傅里叶变换值。
    """
    import math
    return integral(lambda t: func(t) * math.exp(-1j * w * t), -math.inf, math.inf, 1000)

我们准备好了50多个函数

In [203]:
function_descriptions = {
    "Basic_add": "计算两个浮点数的和",
    "Basic_subtract": "计算两个浮点数的差",
    "Basic_divide": "计算两个浮点数的商",
    "Basic_mutiply": "计算两个浮点数的积",
    "Basic_power": "计算一个浮点数的幂",
    "Basic_sqrt": "计算一个浮点数的平方根",
    "Basic_mod": "计算两个浮点数的模",
    "Basic_abs_val": "计算一个浮点数的绝对值",
    "Basic_factorial": "计算一个整数的阶乘",
    "Basic_gcd": "计算两个整数的最大公约数",
    "Basic_lcm": "计算两个整数的最小公倍数",
    "Basic_exp": "计算e的幂",
    "Basic_log": "计算给定基数的对数",
    "Basic_log10": "计算以10为底的对数",
    "Basic_round_val": "对浮点数进行四舍五入",
    "Basic_level_ceil": "向上取整一个浮点数",
    "Basic_level_floor": "向下取整一个浮点数",
    "Tri_func_sin_val": "计算一个角度的正弦值",
    "Tri_func_cos_val": "计算一个角度的余弦值",
    "Tri_func_tan_val": "计算一个角度的正切值",
    "Tri_func_asin_val": "计算一个值的反正弦值",
    "Tri_func_acos_val": "计算一个值的反余弦值",
    "Tri_func_atan_val": "计算一个值的反正切值",
    "Tri_func_sinh_val": "计算一个角度的双曲正弦值",
    "Tri_func_cosh_val": "计算一个角度的双曲余弦值",
    "Tri_func_tanh_val": "计算一个角度的双曲正切值",
    "Tri_func_degrees": "将弧度转换为角度",
    "Tri_func_radians": "将角度转换为弧度",
    "Prob_comb": "计算组合数C(n, k)",
    "Prob_perm": "计算排列数P(n, k)",
    "Prob_factorial_prob": "计算n的阶乘概率",
    "Prob_binom_pmf": "计算二项分布的概率质量函数",
    "Prob_binom_cdf": "计算二项分布的累积分布函数",
    "Prob_poisson_pmf": "计算泊松分布的概率质量函数",
    "Prob_poisson_cdf": "计算泊松分布的累积分布函数",
    "Prob_uniform_pdf": "计算均匀分布的概率密度函数",
    "Prob_uniform_cdf": "计算均匀分布的累积分布函数",
    "Prob_normal_pdf": "计算正态分布的概率密度函数",
    "Prob_normal_cdf": "计算正态分布的累积分布函数",
    "Prob_expon_pdf": "计算指数分布的概率密度函数",
    "Prob_expon_cdf": "计算指数分布的累积分布函数",
    "Prob_geom_pmf": "计算几何分布的概率质量函数",
    "Prob_geom_cdf": "计算几何分布的累积分布函数",
    "Prob_beta_pdf": "计算Beta分布的概率密度函数",
    "Prob_beta_cdf": "计算Beta分布的累积分布函数",
    "Advanced_Math_Derivative": "计算函数在某点的导数",
    "Advanced_Math_integral": "计算函数在给定区间的定积分",
    "Advanced_Math_double_integral": "计算二重积分",
    "Advanced_Math_partial_derivative": "计算函数在某点的偏导数",
    "Advanced_Math_gradient": "计算函数在某点的梯度",
    "Advanced_Math_laplacian": "计算函数在某点的拉普拉斯算子",
    "Advanced_Math_divergence": "计算向量场在某点的散度",
    "Advanced_Math_curl": "计算向量场在某点的旋度",
    "Advanced_Math_jacobian": "计算函数在某点的雅可比矩阵",
    "Advanced_Math_hessian": "计算函数在某点的黑塞矩阵",
    "Advanced_Math_taylor_series": "计算泰勒级数的前n项",
    "Advanced_Math_fourier_series": "计算函数的傅里叶级数",
    "Advanced_Math_laplace_transform": "计算拉普拉斯变换",
    "Advanced_Math_fourier_transform": "计算傅里叶变换"
}

In [204]:
function_names = [
    "Basic_add", "Basic_subtract", "Basic_divide", "Basic_mutiply", "Basic_power", "Basic_sqrt", "Basic_mod", 
    "Basic_abs_val", "Basic_factorial", "Basic_gcd", "Basic_lcm", "Basic_exp", "Basic_log", 
    "Basic_log10", "Basic_round_val", "Basic_level_ceil", "Basic_level_floor", 
    "Tri_func_sin_val", "Tri_func_cos_val", "Tri_func_tan_val", "Tri_func_asin_val", 
    "Tri_func_acos_val", "Tri_func_atan_val", "Tri_func_sinh_val", "Tri_func_cosh_val", 
    "Tri_func_tanh_val", "Tri_func_degrees", "Tri_func_radians", 
    "Prob_comb", "Prob_perm", "Prob_factorial_prob", "Prob_binom_pmf", 
    "Prob_binom_cdf", "Prob_poisson_pmf", "Prob_poisson_cdf", "Prob_uniform_pdf", 
    "Prob_uniform_cdf", "Prob_normal_pdf", "Prob_normal_cdf", "Prob_expon_pdf", 
    "Prob_expon_cdf", "Prob_geom_pmf", "Prob_geom_cdf", "Prob_beta_pdf", "Prob_beta_cdf", 
    "Advanced_Math_Derivative", "Advanced_Math_integral", "Advanced_Math_double_integral", 
    "Advanced_Math_partial_derivative", "Advanced_Math_gradient", "Advanced_Math_laplacian", 
    "Advanced_Math_divergence", "Advanced_Math_curl", "Advanced_Math_jacobian", 
    "Advanced_Math_hessian", "Advanced_Math_taylor_series", "Advanced_Math_fourier_series", 
    "Advanced_Math_laplace_transform", "Advanced_Math_fourier_transform"
]

In [87]:
function_names.__len__()

59

In [205]:
# 函数列表
function_list = [Basic_add, Basic_subtract, Basic_divide, Basic_mutiply, Basic_power, Basic_sqrt, Basic_mod, 
    Basic_abs_val, Basic_factorial, Basic_gcd, Basic_lcm, Basic_exp, Basic_log, 
    Basic_log10, Basic_round_val, Basic_level_ceil, Basic_level_floor, 
    Tri_func_sin_val, Tri_func_cos_val, Tri_func_tan_val, Tri_func_asin_val, 
    Tri_func_acos_val, Tri_func_atan_val, Tri_func_sinh_val, Tri_func_cosh_val, 
    Tri_func_tanh_val, Tri_func_degrees, Tri_func_radians, 
    Prob_comb, Prob_perm, Prob_factorial_prob, Prob_binom_pmf, 
    Prob_binom_cdf, Prob_poisson_pmf, Prob_poisson_cdf, Prob_uniform_pdf, 
    Prob_uniform_cdf, Prob_normal_pdf, Prob_normal_cdf, Prob_expon_pdf, 
    Prob_expon_cdf, Prob_geom_pmf, Prob_geom_cdf, Prob_beta_pdf, Prob_beta_cdf, 
    Advanced_Math_Derivative, Advanced_Math_integral, Advanced_Math_double_integral, 
    Advanced_Math_partial_derivative, Advanced_Math_gradient, Advanced_Math_laplacian, 
    Advanced_Math_divergence, Advanced_Math_curl, Advanced_Math_jacobian, 
    Advanced_Math_hessian, Advanced_Math_taylor_series, Advanced_Math_fourier_series, 
    Advanced_Math_laplace_transform, Advanced_Math_fourier_transform]

# 获取函数描述的 JSON 字典
math_tools = auto_functions(function_list)

# 将函数列表转换为字典
available_math_functions = {func.__name__: func for func in function_list}

### 3.2 问题解决

**分割 → 粗略对齐 → 精准对齐**

分割

通过分割、粗略对齐、过滤的方式，一步步缩小可能被需要的函数的集合，最终在一个较小的函数集合中实现与普通Function Call一样的对齐。具体来说——

1. **分层架构指导函数分割**
> 在实际商业场景当中，在我们构建复杂的电商app或者金融app时，我们会对架构进行分层，将系统划分为多个层次，每个层次处理特定的功能。常见的分层包括：
> - 表示层（Presentation Layer）：用户接口和用户交互逻辑（注册、登录、更新用户信息等函数功能及应该归属于这一层）。<br><br>
> - 业务逻辑层（Business Logic Layer）：处理应用程序的核心逻辑（订单管理，如创建订单、更新订单状态、查询订单等应该在这一层，商品管理，如添加商品、更新商品信息、查询商品库存等也应该在这一层）。<br><br>
> - 数据访问层（Data Access Layer）：与数据库或外部数据源交互。<br><br>
> - 服务层（Service Layer）：提供系统内部或外部的服务接口。<br><br>
> 无论整体的函数库有多复杂，我们始终是能够按照功能或架构规律按函数分割成更小的子集的、让每个设计层包含相对较少的函数。在实际进行Function Call时，我们先通过模型进行高层次的意图识别，确定使用哪一层的函数，然后再进行更细粒度的选择。通常来说，根据架构分层后、每个层下不会超过1-200个函数。

2. **利用关键词指导函数分割**
 > 如果你的Json Schema写得比较清晰、比较标准，你可以尝试使用关键词先将超大函数库分割成相应的小集合。例如在教育领域，你可能很容易就从“章节”、“年级”这些天然分层中分出较小的函数集合。


3. **更大的函数库所使用的手段**
> 结合向量数据库 + RAG技术，通过语义搜索来快速匹配相关函数。流程如下：
>> 预处理：将每个函数的描述向量化，并存储在向量数据库中。<br><br>
>> 意图向量化：将用户输入向量化，并在向量数据库中进行语义搜索。<br><br>
>> 返回结果：根据搜索结果动态加载相关函数描述。

In [95]:
function_list

[<function __main__.Basic_add(a: float, b: float) -> float>,
 <function __main__.Basic_subtract(a: float, b: float) -> float>,
 <function __main__.Basic_divide(a: float, b: float) -> float>,
 <function __main__.Basic_mutiply(a: float, b: float) -> float>,
 <function __main__.Basic_power(a: float, b: float) -> float>,
 <function __main__.Basic_sqrt(a: float) -> float>,
 <function __main__.Basic_mod(a: float, b: float) -> float>,
 <function __main__.Basic_abs_val(a: float) -> float>,
 <function __main__.Basic_factorial(a: int) -> int>,
 <function __main__.Basic_gcd(a: int, b: int) -> int>,
 <function __main__.Basic_lcm(a: int, b: int) -> int>,
 <function __main__.Basic_exp(a: float) -> float>,
 <function __main__.Basic_log(a: float, base: float = 10) -> float>,
 <function __main__.Basic_log10(a: float) -> float>,
 <function __main__.Basic_round_val(a: float, digits: int = 0) -> float>,
 <function __main__.Basic_level_ceil(a: float) -> int>,
 <function __main__.Basic_level_floor(a: float)

1. **分割**

In [206]:
#如果函数是按顺序排列的，你只需要把它们索引出来、再输入到generate_json中就好
basic_math_tools = auto_functions(function_list[:16])
Tri_func_math_tools = auto_functions(function_list[16:27])
Prob_tools = auto_functions(function_list[27:44])
Advanced_math_tools = auto_functions(function_list[44:])
#也有无法识别和分割的情况，我们同样需要准备好全部函数的JSON description
math_tools = auto_functions(function_list)

In [207]:
available_math_functions = {func.__name__: func for func in function_list}

In [208]:
# 可以通过下面的字典来进行相应的函数集合的调用
function_type = {"基础数学": basic_math_tools
                ,"三角函数": Tri_func_math_tools
                ,"概率论": Prob_tools
                ,"高等数学": Advanced_math_tools
                ,"其他": math_tools}

2. **意图识别 + 精准对齐**

In [105]:
def math_calculate(prompt, function_type, available_functions):
    """
    首先，先对用户的意图进行识别，判断是哪一类问题
    """
    prompt = prompt
    judge_prompt = "请问上述问题最接近哪一类数学问题？\
    请回应基础数学、或三角函数、或概率论、或高等数学，或其他。\
    不要回应别的任何文字。"
    message = [
        {"role": "user", "content": prompt + judge_prompt}
    ]

    #进行初次意图识别
    response_first_align = client.chat.completions.create(
    model="glm-4",
    messages=message #此时其实最好的方式是使用function_call而不是prompt
    )

    #根据意图识别出的数据类别，定位相应类别的函数群
    #在这里进行关键字匹配
    func_type = response_first_align.choices[0].message.content
    current_function_tools = None

    if "基础数学" in func_type:
        current_function_tools = function_type["基础数学"]
    elif "三角函数" in func_type:
        current_function_tools = function_type["三角函数"]
    elif "概率论" in func_type:
        current_function_tools = function_type["概率论"]
    elif "高等数学" in func_type:
        current_function_tools = function_type["高等数学"]
    else:
        current_function_tools = function_type["其他"]

    print("初步意图识别结果:{}".format(func_type),"\n")

    #确认函数群后，直接使用原始prompt在函数群中执行function call
    message = [{"role":"assistant","content":"当你被问到数学问题时，\
                                              尝试调用Tools来解决问题。"}]
    message.append({"role": "user", "content": prompt})

    response_f = client.chat.completions.create(
        model="glm-4",  
        messages=message,
        tools = current_function_tools,
        tool_choice = "auto"
        )
    
    try:
        #是否有成功调用函数？
        function_calls = response_f.choices[0].message.tool_calls[0]
        print(" ")
        print("\n被调用的函数是{}".format(response_f.choices[0].message.tool_calls[0].function.name))
        
        function_to_call = available_functions[function_calls.function.name]
        function_args = json.loads(function_calls.function.arguments)
        function_response = function_to_call(**function_args)

        message = []
        message.append({
                "role": "assistant",
                "content": "你使用了tools工具，最终获得的答案是" + str(function_response),
            })
        message.append({"role": "user", "content": prompt + "请仅仅回答题目的答案（如果是数字，则只显示数字），\
                                                             不要包括其他描述"})
        
        response_a = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=message
            )
    
        print(response_a.choices[0].message.content)

    except:
        print(" ")
        print("\n没有函数被成功调用！请重新给解决方案")
        response_f = client.chat.completions.create(
            model="glm-4",  
            messages=message,
            tools = current_function_tools,
            tool_choice = "auto"
        )

In [102]:
1314520 / 520

2527.923076923077

In [103]:
prompt = "请帮我计算一下 1314520 除以 520 等于几?"

math_calculate(prompt, function_type, available_math_functions)

初步意图识别结果:基础数学

计算结果是：
1314520 ÷ 520 = 2520 

 

被调用的函数是Basic_divide
2527.923076923077


In [106]:
prompt = "求函数 \( f(x) = e^x \) 在 \( x = 0 \) 处的泰勒展开式，并利用该展开式近似计算 \( e^{0.2} \) 的值。"

math_calculate(prompt, function_type, available_math_functions)

初步意图识别结果:高等数学 

 

没有函数被成功调用！请重新给解决方案


 **我们分析为什么会调用失败？？**

- 测试1：当函数库比较短，且不存在相应可调用的函数时：

In [109]:
prompt = "求函数 \( f(x) = e^x \) 在 \( x = 0 \) 处的泰勒展开式，并利用该展开式近似计算 \( e^{0.2} \) 的值。\
          其中e是自然底数exp"
current_function_tools = Advanced_math_tools[:4]
message=[]
message.append({"role": "user", "content": prompt})

response_f = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    tools = current_function_tools,
    tool_choice = "auto",
    temperature = 0.1
    )

In [110]:
response_f

Completion(model='glm-4', created=1718212480, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8741741654316771050', function=Function(arguments='{}', name='Advanced_Math_Derivative'), type='function', index=0)]))], request_id='8741741654316771050', id='8741741654316771050', usage=CompletionUsage(prompt_tokens=1824, completion_tokens=33, total_tokens=1857))

发现大模型帮我们识别出来需要调用Advanced_Math_Derivative函数，虽然函数调用错误，但是这个时候大模型还是能调用大模型

- 测试2：当函数库比较长，但是不存在可以调用的函数时

In [209]:
# 要剔除的函数名称
names_to_remove = ['Advanced_Math_taylor_series', 'Advanced_Math_taylor_expansion']

# 过滤列表，移除包含指定函数名称的字典
Advanced_math_tools_without_taylor = [item for item in Advanced_math_tools if item['function']['name'] not in names_to_remove]

In [210]:
prompt = "求函数 \( f(x) = e^x \) 在 \( x = 0 \) 处的泰勒展开式，并利用该展开式近似计算 \( e^{0.2} \) 的值。\
          其中e是自然底数exp"
current_function_tools = Advanced_math_tools_without_taylor

message=[]
message.append({"role": "user", "content": prompt})

response_f = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    tools = current_function_tools,
    tool_choice = "auto",
    temperature = 0.1
    )

In [211]:
response_f

Completion(model='glm-4', created=1718286622, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='Advanced_Math_derivative\n```python\ntool_func = lambda x: exp(x)\ntool_a = 0\ntool_h = 0.001\n```', role='assistant', tool_calls=None))], request_id='8741750759647600659', id='8741750759647600659', usage=CompletionUsage(prompt_tokens=4496, completion_tokens=33, total_tokens=4529))

In [212]:
response_f.choices[0].message.content

'Advanced_Math_derivative\n```python\ntool_func = lambda x: exp(x)\ntool_a = 0\ntool_h = 0.001\n```'

发现当我们的函数变多的时候，没有调用外部函数，而是把函数当做了语料库了

- 测试3：询问信息是否在知识库里

In [215]:
prompt = "求函数 \( f(x) = e^x \) 在 \( x = 0 \) 处的泰勒展开式，并利用该展开式近似计算 \( e^{0.2} \) 的值。\
          其中e是自然底数exp" + "请问这个问题在你的知识库里可以解答吗？"
current_function_tools = Advanced_math_tools
message=[]
message.append({"role": "user", "content": prompt})

response_f = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    tools = current_function_tools,
    tool_choice = "auto",
    temperature = 0.1
    )

In [216]:
response_f

Completion(model='glm-4', created=1718286751, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='Advanced_Math_taylor_series\n```python\ntool_func(func=lambda x: exp(x), a=0, n=5)\n```', role='assistant', tool_calls=None))], request_id='8741752099677470505', id='8741752099677470505', usage=CompletionUsage(prompt_tokens=4964, completion_tokens=29, total_tokens=4993))

In [217]:
response_f.choices[0].message.content

'Advanced_Math_taylor_series\n```python\ntool_func(func=lambda x: exp(x), a=0, n=5)\n```'

感觉好像把我们的函数库当做知识库使用了

**尝试解决**

In [122]:
prompt = "求函数 \( f(x) = e^x \) 在 \( x = 0 \) 处的泰勒展开式，并利用该展开式近似计算 \( e^{0.2} \) 的值。\
          其中e是自然底数exp" + "请按照 '调用函数名称'\n'''python\ntool_func(函数所需参数)\n'''的方式返回结果。"
current_function_tools = Advanced_math_tools

message = [{"role":"assistant","content":"当你被问到数学问题时，\
                                          尝试调用Tools来解决问题。"}]
message.append({"role": "user", "content": prompt})

response_f = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    tools = current_function_tools,
    tool_choice = "auto",
    temperature = 0.1
    )

In [123]:
response_f

Completion(model='glm-4', created=1718213349, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='Advanced_Math_taylor_series\n```python\ntool_func(func=lambda x: exp(x), a=0, n=5)\n```', role='assistant', tool_calls=None))], request_id='8741741757396012009', id='8741741757396012009', usage=CompletionUsage(prompt_tokens=4993, completion_tokens=29, total_tokens=5022))

In [173]:
prompt = prompt + response_f.choices[0].message.content
message=[]
message.append({"role": "user", "content": prompt + "检查上述计算过程中可能使用到的python库或工具，\
                                                    并将这些工具全部使用import导入\
                                                    例如```python import math from sympy import exp```\
                                                    只返回代码，不要返回任何其他说明或信息"})

result = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    temperature = 0.1
    )

In [174]:
result

Completion(model='glm-4', created=1718215566, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='```python\nimport math\nfrom sympy import exp, symbols, series\n```', role='assistant', tool_calls=None))], request_id='8741741894834996616', id='8741741894834996616', usage=CompletionUsage(prompt_tokens=210, completion_tokens=19, total_tokens=229))

In [175]:
print(result.choices[0].message.content)

```python
import math
from sympy import exp, symbols, series
```


In [176]:
# 定义清理字符串的函数
def clean_string(input_string):
    # 删除所有不可见的控制字符
    cleaned_string = re.sub(r'[\x00-\x1F\x7F]', '', input_string)
    # 将所有换行符标准化为 \n
    cleaned_string = cleaned_string.replace('\r\n', '\n').replace('\r', '\n')
    return cleaned_string

In [177]:
clean_content = clean_string(response_f.choices[0].message.content)

In [178]:
print(clean_content)

Advanced_Math_taylor_series```pythontool_func(func=lambda x: exp(x), a=0, n=5)```


In [179]:
import re
import math
# 给定的字符串
code_str = clean_content
import_code_str = result.choices[0].message.content

# 提取Python代码段的正则表达式模式
code_pattern_core = re.compile(r'```python(.*?)```', re.DOTALL)

# 提取code_str中的Python代码段
match_code = code_pattern_core.search(code_str)
if match_code:
    python_code = match_code.group(1)

# 提取import_code_str中的Python代码段
match_import = code_pattern_core.search(import_code_str)
if match_import:
    import_code = match_import.group(1)
else:
    import_code = import_code_str  # 如果import_code_str没有代码块，就直接使用整个字符串

# 替换'tool'为'Advanced_Math_taylor_expansion'以便执行
python_code = python_code.replace('tool_func', 'result = Advanced_Math_taylor_expansion')

# 合并import_code和python_code
combined_code = import_code + '\n' + python_code

print(combined_code)

# 执行合并后的代码
local_vars = {}
exec(combined_code, globals(), local_vars)

# 获取执行结果
math_result = local_vars.get('result')
print(math_result)


import math
from sympy import exp, symbols, series

result = Advanced_Math_taylor_expansion(func=lambda x: exp(x), a=0, n=5)
x**5/120 + x**4/24 + x**3/6 + x**2/2 + x + 1


## 4. 挑战3-函数并发调用

- 串行调用的场景

In [195]:
prompt = "请问1234567与7654321的和与积作差等于几？" # Agent
message=[]
message.append({"role": "user", "content": prompt + "解决这个问题要分几步？\
                                                     记住只告诉我步骤，不要解决问题，也不要说额外的话\
                                                     请按照'1. xxx;2. xxx;...;的格式列举\
                                                     步骤与步骤之间使用半角分号;相隔，整个句子请用半角分号;结尾"})

response_step = client.chat.completions.create(
    model="glm-4",  
    messages=message,
    temperature = 0.1
    )

In [196]:
response_step

Completion(model='glm-4', created=1718216123, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='1. 计算两个数的和: 将1234567与7654321相加; \n2. 计算两个数的积: 将1234567与7654321相乘; \n3. 计算和与积的差: 将步骤1得到的和减去步骤2得到的积; \n4. 得出最终结果: 得到步骤3计算出的差值。 \n\n整个解决问题过程分为4步。', role='assistant', tool_calls=None))], request_id='8741747907789403382', id='8741747907789403382', usage=CompletionUsage(prompt_tokens=88, completion_tokens=98, total_tokens=186))

In [197]:
print(response_step.choices[0].message.content)

1. 计算两个数的和: 将1234567与7654321相加; 
2. 计算两个数的积: 将1234567与7654321相乘; 
3. 计算和与积的差: 将步骤1得到的和减去步骤2得到的积; 
4. 得出最终结果: 得到步骤3计算出的差值。 

整个解决问题过程分为4步。


In [198]:
str_content=response_step.choices[0].message.content

In [199]:
# 定义正则表达式模式
pattern = r'(.*?;)'

# 使用re.findall提取所有匹配项
steps = re.findall(pattern, str_content)

# 去除最后一个不完整的匹配项
steps = [step.strip() for step in steps if step.strip()]

for step in steps:
    print(step)

1. 计算两个数的和: 将1234567与7654321相加;
2. 计算两个数的积: 将1234567与7654321相乘;
3. 计算和与积的差: 将步骤1得到的和减去步骤2得到的积;


In [200]:
step_answer = ""
# 步骤之间相互独立或者有联系，都可以用
for step in steps:
    message = [{"role":"assistant","content":prompt + "正在分步骤解决问题。已知条件是" + step_answer}]
    message.append({"role":"user","content":"当前步骤是" + step})

    response_per_step = client.chat.completions.create(
        model="glm-4",  
        messages=message,
        tools = basic_math_tools,
        tool_choice = True,
        temperature = 0.1
        )

    try:
        function_calls = response_per_step.choices[0].message.tool_calls[0]
        function_to_call = available_math_functions[function_calls.function.name]
        function_args = json.loads(function_calls.function.arguments)
        function_response = function_to_call(**function_args)
    except:
        function_response = response_per_step.choices[0].message.content

    step_answer = step + "答案是" + str(function_response)
    print(step_answer)

#经历完整的步骤、获得答案之后再返回
message = []
message.append({
        "role": "assistant",
        "content": "你使用了tools工具，最终获得的答案是" + str(function_response),
    })
message.append({"role": "user", "content": prompt + "请仅仅回答答案的数字，不要包括其他描述"})

response_a = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=message
    )

print(response_a.choices[0].message.content)

1. 计算两个数的和: 将1234567与7654321相加;答案是8888888
2. 计算两个数的积: 将1234567与7654321相乘;答案是9449772114007
3. 计算和与积的差: 将步骤1得到的和减去步骤2得到的积;答案是-6419754
-6419754


- 并行调用的场景

In [ ]:
prompts = """添加如下用户信息：
1. {name:"zhangsan",age:33}
2. {name:"lisi",age=44}
3. {name:"wangwu",age=55}
4. {name:"zhaoliu",age=66}
"""

**LangChain实现的方式**

In [ ]:
messages1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是机器学习"),]

In [ ]:
messages2 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是AIGC"),]

In [ ]:
messages3 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是大模型技术"),]

In [ ]:
reponse = chat.batch([messages1,
            messages2,
            messages3,])

reponse

**也可以使用异步调用的方式**

In [ ]:
reponse1 = await chat.ainvoke(messages1)
reponse2 = await chat.ainvoke(messages2)
reponse3 = await chat.ainvoke(messages3)
reponse4 = await chat.ainvoke(messages4)

**GLM batch调用**

In [ ]:
{
      "custom_id": "request-1",  #每个请求必须包含custom_id且是唯一的，用来将结果和输入进行匹配
      "method": "POST",
      "url": "/v4/chat/completions", 
      "body": {
          "model": "glm-4",     #每个batch文件只能包含对单个模型的请求,支持 glm-4、glm-3-turbo.
          "messages": [
              {"role": "system","content": "你是一个意图分类器."},
              {"role": "user", "content": """
              # 任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，
              # 评论：review = "订单处理速度太慢，等了很久才发货。"
              # 输出格式：
              {
              "分类标签": " ", 
              "特定问题标注": " " 
              }
              """
              }
          ],
          "temperature": 0.1
      }
  }

In [ ]:
{"custom_id": "request-1", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"订单处理速度太慢，等了很久才发货。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-2", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \",商品有点小瑕疵，不过客服处理得很快，总体满意。\",# 输出格式：'''{\",分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-3", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"这款产品性价比很高，非常满意。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-4", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"说明书写得不清楚，看了半天也不知道怎么用。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-5", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"总体还不错，但价格偏高，不太划算。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-6", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"物流速度很慢，等了两个星期才收到货 \"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-7", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"收到的产品跟描述不符，有些失望。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-8", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"客服很耐心，解决问题很快，感谢！\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-9", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"包装太简单，商品在运输过程中被压坏了。\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}
{"custom_id": "request-10", "method": "POST", "url": "/v4/chat/completions", "body": {"model": "glm-4", "messages": [{"role": "system", "content": "你是一个意图分类器."},{"role": "user", "content": "#任务：对以下用户评论进行情感分类和特定问题标签标注，只输出结果，# 评论：review = \"产品质量不错，但是颜色和图片上的不一样\"# 输出格式：'''{\"分类标签\": \" \", \"特定问题标注\": \" \" } '''"}]}}

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="") # 请填写您自己的APIKey
  
result = client.files.create(
    file=open("product_reviews.jsonl", "rb"),
    purpose="batch"
)
print(result.id)

In [ ]:
#创建batch任务

from zhipuai import ZhipuAI
 
client = ZhipuAI()  # 填写您自己的APIKey
 
create = client.batches.create(
    input_file_id="file_123",
    endpoint="/v4/chat/completions", 
    completion_window="24h", #完成时间只支持 24 小时
    metadata={
        "description": "Sentiment classification"
    }
)
print(create)

In [ ]:
#打印batch流程

batch_job = client.batches.retrieve("batch_id")
print(batch_job)

## 5. 挑战四-响应慢

- 特定场景下的缓存方案设计

- 把前面三步做好，某种意义上来讲也是一种提升响应

更多方案期待大家分享